In [69]:
%load_ext autoreload
%autoreload 2  
%reload_ext autoreload
import sys
print(sys.executable)
sys.path.append('..')
import pickle
from pathlib import Path
from rich import print
from rich.console import Console
import pdb
import json
import random
import copy
import time
import re
from ruamel.yaml import YAML 
console = Console()

# Import your modules
from dialop.evaluate import CheckpointManager
from dialop.responses_class import ParallelConversations, ResponseManager, Conversation
from dialop.envs.planning import PlanningEnv
from skills import Agent_tell
from dialop.games.planning import PlanningGame
print("Imports successful!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/georgiazhou/research_machine/dialop/.venv/bin/python

Imports successful!

In [70]:

import gspread
from google.oauth2.service_account import Credentials

SERVICE_ACCOUNT_FILE = '/Users/georgiazhou/research_machine/dialop/dialop-8759580d9f40.json'
SCOPE = ["https://spreadsheets.google.com/feeds", 
         "https://www.googleapis.com/auth/drive"]
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPE)
client = gspread.authorize(credentials)
spreadsheet = client.open('Dialop-GPT4-Planning')
worksheet = spreadsheet.get_worksheet(8)

In [4]:
# Initialize checkpoint manager


/Users/georgiazhou/research_machine/dialop/dialop/checkpoints/debug_states

Successfully loaded checkpoint!

Turn: 2

Features: ['', '{AtLeastOneEventType: landmark, rating: good, has_parking: true}']

Players: ['agent', 'user']

In [5]:
# Access player objects



=== Agent Info ===

Agent role: agent

=== User Info ===

User role: user

User observations: 
You:Travel Preferences:
go to at least one landmark
renting a car so preferably places with parking
definitely want to check out Dan's recommendations: Central Plaza, The Mall
prefer ratings above 4
generally like eating vegan options
check out live music!
family trip, best to go to places that are good for kids
find a place with wifi to do some work at some point
keep budget below $40
COVID conscious, outdoor seating places would be best
minimize travel distance
you must follow your communication style:Communication Style:
Feedback Provider: This client is likely to give feedback and suggestions throughout the planning process and 
appreciates being asked for their opinions.
Detail-Oriented Client: This client pays attention to the finer details and may ask specific questions about 
logistics, timing, and contingency plans.
You:
You:I'm planning to visit with my family and would prefer a good rated location. Dan recommended Central Plaza and 
The Mall. By the way, do these places have parking available?
Error: Invalid message: I'm planning to visit with my family and would prefer a good rated location. Dan 
recommended Central Plaza and The Mall. By the way, do these places have parking available?.Messages must be 
formatted with a type like '<content here>'
You: My main focus is to visit at least one landmark with a good rating, check out Dan's recommendations including 
Central Plaza, and The Mall, and secure a place with parking as I am renting a car. Can you suggest places 
according to these preferences?

In [170]:
checkpoint_mgr = CheckpointManager()

        # Load the initial state
initial_state = checkpoint_mgr.load_full_state("initial_state")
try:
    # Access key components
    players = initial_state['players']
    extracted_features = initial_state.get('extracted_features', [])
    t = initial_state.get('t', 0)
    env = initial_state.get('env', None)
    agent = initial_state['players']['agent']
    user = initial_state['players']['user']
    chat_history = agent.prompt
    if not env:
        print("No environment found in checkpoint.")
    print(f"\nTurn: {t}")
    print(f"Features: {extracted_features}")
    print(f"Players: {list(players.keys())}")
    
except Exception as e:
    print(f"Error loading checkpoint: {str(e)}")

Loaded .api_key
Loaded .api_key
Loaded .api_key


Turn: 6

Features: {'has_parking': ['not a deal breaker', 'Low'], 'WantToGo': ['The Mall', 'Unknown'], 
'AtLeastOneEventType': ['landmark', 'Unknown']}

Players: ['agent', 'user']

In [139]:
def n_different_responses(players, strategy_response_list, t):
    
    
    resp1 = players['agent'].respond(t, 35, vary=True, strategy = strategy_response_list[0])
    resp2 = players['agent'].respond(t, 35, vary=True, strategy = strategy_response_list[1])
    resp3 = players['agent'].respond(t, 35, vary=True, strategy = strategy_response_list[2])
    return [resp1, resp2, resp3]




In [136]:



def initialize_parallel_conversations(agent_3_responses, strategy_response_list, players, env, t):
    # Create parallel conversation manager
    parallel_convs = ParallelConversations(
        num_streams=3,
        players=players,
        env_ctor=lambda: PlanningEnv(),
        env=env,
        max_length=35
    )
    
    # Generate initial responses
    
    # Initialize parallel streams
    parallel_convs.initialize_streams(agent_3_responses, strategy_response_list, t)
    
    print("\nInitialized parallel conversations:")
    for i, conv in enumerate(parallel_convs.conversations):
        state = conv.get_current_state()
        print(f"\nStream {i+1}:")
        print(f"Initial response: {agent_3_responses[i][:100]}...")
        print(f"Features extracted: {state.features}")
        
    return parallel_convs

# Initialize parallel conversations




Loaded .api_key
Loaded .api_key
Loaded .api_key


In [73]:
def style_judger(parallel_convs):
    with open ("/Users/georgiazhou/research_machine/dialop/dialop/RL/explanation_per_persona.txt", "r") as f:
        explanation_per_persona = json.load(f)
    user_base_prompt = parallel_convs.c_players['user'].user_prompt_obss
    first_you = user_base_prompt.find("You:")
    second_you = user_base_prompt.find("You:", first_you + 1)
    user_base_prompt = user_base_prompt[second_you:]
    prompt = "You are a communication expert. Please first judge whether the following messages on conform to one of the styles listed. You should only confirm if you are very confident. If you don't think the message conforms to any of the styles, output: {}. If you find one or two that match, output the style(s) that you think it matches as {style : explanation, ...}. Here is conversation history: " + "\n" + user_base_prompt + "Here are the styles you should judge against: " + "\n" + str(explanation_per_persona) + "Remember you should only output the style(s) that you are very confident about. If you are not very confident about any of the options, output: {}.  "
    response = Agent_tell(prompt)
    
    return response



In [119]:
def get_conversation_history(game):
    # Full action log (includes messages, proposals, and responses)
    full_history = game.action_log  # List of dicts with type, message, player, time
    
    # Messages only
    messages_only = game.message_history  # List of message strings
    
    # Format by player
    all_conversation = ''
    for action in game.action_log:
        if action["type"] == "message":
            speaker = "agent" if action["player"] == 1 else "user"
            conversation_str = f"{speaker}: {action['message']}\n"
            all_conversation += conversation_str
    
    return all_conversation

In [155]:
def strategy_sampling(env):
    with open("/Users/georgiazhou/research_machine/dialop/dialop/envs/data/planning_strategy_sampling.txt", "r") as f:
        strategy_prompt = f.read()
    
    chat_history = get_conversation_history(env.game)

    whole_prompt = strategy_prompt + "\n" + chat_history
    
    
    strategy_response = Agent_tell(whole_prompt)
    strategy_response = re.sub(r'\n\s*\n', '\n', strategy_response)
    strategy_response_list = strategy_response.split('\n')

    
    return strategy_response_list



In [74]:

def step_parallel_conversations(parallel_convs):
    """Step conversations until each hits a user turn"""
    
    # Track which conversations are waiting for user input
    waiting_for_user = [False] * len(parallel_convs.conversations)
    
    
    while not all(waiting_for_user):
        # Get responses for conversations where it's agent's turn
    
        active_convs = {0:'', 1:'', 2:''}  # Track which conversations get stepped
        
        for i, conv in enumerate(parallel_convs.conversations):
            if waiting_for_user[i]:
                active_convs[i] = None  # Placeholder to maintain indexing
                continue
                
            state = conv.get_current_state()
            if state.obss["turn_player"] == "agent":
                response = conv.players["agent"].respond(vary=True)
                active_convs[i] = response
            
            else:
                # Hit user turn - mark this conversation as waiting
                waiting_for_user[i] = True
                active_convs[i] = None
                print(f"----Conversation {i} is waiting for user input---")
        
        # Step only the active conversations forward
        if active_convs:

            print("\nStepping conversations:", active_convs)
            for i, response in active_convs.items():
                if response is not None:
                    print(f"\nConversation {i} response:\n{response[:100]}...")
                    conv = parallel_convs.conversations[i]
                    players = parallel_convs.players_dict[i]
                    print(response)
                    
                    conv.step(str(response), conv.get_current_state().turn + 1)
                    [player.observe(conv.state.obss[pname]) for pname, player in players.items()]
                
        
    
    print("\nAll conversations are now waiting for user input")
    # Print current state of each conversation
    for i, conv in enumerate(parallel_convs.conversations):
        state = conv.get_current_state()
        print(f"\nConversation {i} (Turn {state.turn}):")
        print(f"Last observation: {state.obss.get('user', '')[:100]}...")
    
    return parallel_convs

# Use it like this:



In [75]:
def get_conversation_streams_spreadsheet(parallel_convs):
    """Get the full conversation history for each parallel stream"""
    conversation_streams = []
    
    for i, conv in enumerate(parallel_convs.conversations):
        stream = []
        # Get all observations from conversation history
        for state in conv.history:
            if 'user' in state.obss:
                stream.append(('user', state.obss['user']))
            if 'agent' in state.obss:
                stream.append(('agent', state.obss['agent']))
                # Get all observations from conversation history
        formatted_stream = '\n'.join([f"{speaker}: {message}\n" for speaker, message in stream])     
        worksheet.update_cell(2, 8+i, formatted_stream)
        conversation_streams.append(stream)

             
    # Save conversation streams to spreadsheet

            
    # Update cell H2 with the conversation streams
    
            
    return conversation_streams

# Get the conversation streams



In [9]:

def clean_user_conversation(base_user_prompt_obss,  parallel_convs):
    user_conversations_no_pref = []
    
    for i in range(3):
        user_conve_record = base_user_prompt_obss + parallel_convs.players_dict[i]['user'].user_prompt_obss
        first_you = user_conve_record.find("You:")
        second_you = user_conve_record.find("You:", first_you + 1)

    # Get everything from second "You:" onwards
        user_conversations_no_pref.append(user_conve_record[second_you:])
    return user_conversations_no_pref





In [62]:

def sample_preferences(known_preferences):
    potential_prefs_list = json.loads(open("/Users/georgiazhou/research_machine/dialop/dialop/RL/feature_dict.txt").read())
    # it's a dictionary, so we need to get the keys; known should also be a dictionary
    # make sure known preferences are keys or dicts/
    available_prefs = list(set(potential_prefs_list.keys()) - set(known_preferences))
    pref_list = random.sample(available_prefs, (10-len(known_preferences)))
    pref_dict = {}
    for pref in pref_list:
        if potential_prefs_list[pref] == 'binary':
            pref_dict[pref] = random.choice(['like', 'dislike'])
        else:
            pref_dict[pref] = 'like'
        
    return pref_dict
def get_column_letter(n):
    """Convert number to Excel column letters (A->1, B->2, AA->27, etc)"""
    if n == 0:
        return 'A'
    while n > 0:
        n, remainder = divmod(n, 26)
        result = chr(65 + remainder) 
    print(result)
    return result

global_preferences_only = []
def n_1_user_personas(formatted_sampled_preferences, extracted_features, record = None, style_response = None):
    """
    Create 10 user personas for each conversation; no longer use sample_preferences
    """

    global global_preferences_only
    if re.search('Communication Style', formatted_sampled_preferences):
        formatted_sampled_preferences = formatted_sampled_preferences[:formatted_sampled_preferences.find('Communication')]
    # currently there's 1 user per agent reply. 
    #for j,record in enumerate(user_conversations_no_pref):
    #prompts = []
    #preferences = []
    #for i in range(10):
    # If formatted_sampled_preferences is a list of strings, join them with newlines
    if isinstance(extracted_features, dict):
        formatted_features = []
        for key, value in extracted_features.items():
            if value[1] == 'Unknown':
                importance = random.choice(['High', 'Medium', 'Low'])
                formatted_features.append(f"{key}, {value[0]} : importance {importance}")
            else:
                formatted_features.append(f"{key}, {value[0]} : importance {value[1]}")
        extracted_features = '\n'.join(formatted_features)
    
    if style_response:
        preference = 'Here are your preferences: ' + str(formatted_sampled_preferences) + str(extracted_features)  + "\n" + "Here are your communication styles: " + str(style_response)
    else:
        preference = 'Here are your preferences: ' +  str(formatted_sampled_preferences) + str(extracted_features) 
        
                
    if record:
        features_prompt = preference + '\n' + record
    else:
        features_prompt = preference
    
    global_preferences_only.append(str(formatted_sampled_preferences) + str(extracted_features))
    #prompts.append(features_prompt)
    #preferences.append(preference)

    #worksheet.update(f'{get_column_letter(10*j)}{5}:{get_column_letter(10*j + 9)}{5}', [preferences])

        
    return features_prompt




# do I sample communication style? 

# next I should make sure info used to create users are helpful (what cuisine, what distance, etc). Also make sure the 
# user understands the preferences. Then I add the general user prompt to the begining of each user prompt

In [11]:
def create_10_new_user_per_conv(half_prompts, parallel_convs):
    players = {}
    for i, half_per_conv in enumerate(half_prompts):
        players[i] = parallel_convs.create_pair_new_players(parallel_convs.players_dict[i], multiple_user_personas_incomplete_prompt=half_per_conv)
    return players
    


#print(agent_user_10_pairs_dict[0]['user_0'].prompt, agent_user_10_pairs_dict[0]['user_1'].prompt)


In [77]:
def format_user_game_state(base_env, incomplete_action_log = False):
    """create custom prefereces and score function (see game class)"""
    if incomplete_action_log:
        action_log = []
    else:
        action_log = base_env.game.action_log
    game_state = {
                "action_log": action_log,
                "events": base_env.game.events,
                "preferences": base_env.game.prefs,  # Must be in the format [(desc, weight, cls_name, cls_args), ...]
                "persona_styles": base_env.game.persona_styles
                }
    return game_state


In [76]:
def parse_extracted_features(extracted_features):
    parsed_features = []
    for f in extracted_features:
        parsed = parse_feature_string(f)
        if parsed:
            parsed_features.extend(parsed)
    return parsed_features
def parse_feature_string(feature_str):
    """Convert feature string to list of features."""
    if not feature_str or feature_str == '':
        return []
    
    # Remove outer braces and split on commas
    feature_str = feature_str.strip('{}')
    features = [f.strip() for f in feature_str.split(',')]
    
    # Clean up each feature
    parsed = []
    for f in features:
        if f:  # Skip empty strings
            parsed.append(f.strip())
            
    return parsed

# Parse extracted features





In [171]:


strategy_response_list = strategy_sampling(env)





In [157]:
print(len(strategy_response_list))


3

In [172]:
agent_3_responses = n_different_responses(players, strategy_response_list, t)

Response:  [message] Thanks for your response! Central Plaza and The Mall will certainly be part of our plan. Are 
there any specific landmarks or activities you'd like to consider in our plan, or other deal-breakers to keep in 
mind?

stop:  stop

CompletionUsage(
    completion_tokens=47,
    prompt_tokens=2130,
    total_tokens=2177,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

Response:  [tool] Search(fields=[name, category, has parking, distance_to(Central Plaza)], filters=[category == 
landmark, has parking == True], sort_by=[distance_to(Central Plaza)], limit=1)

stop:  stop

CompletionUsage(
    completion_tokens=43,
    prompt_tokens=2121,
    total_tokens=2164,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

Response:  [tool] Search(fields=[name, category, has parking], filters=[category == landmark], 
sort_by=[distance_to(The Mall)], limit=1)

stop:  stop

CompletionUsage(
    completion_tokens=31,
    prompt_tokens=2125,
    total_tokens=2156,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

In [173]:
parallel_convs = initialize_parallel_conversations(agent_3_responses, strategy_response_list, players, env, t)


{'user': "\nAgent: [message] Thanks for your response! Central Plaza and The Mall will certainly be part of our plan. Are there any specific landmarks or activities you'd like to consider in our plan, or other deal-breakers to keep in mind?", 'agent': '', 'turn_player': 'user', 'done': False, 'reward': 0, 'info': {'num_msgs': 3}}
ChatCompletion(id='chatcmpl-ATlAI08gJSDe8j7tlwssB9bVJS3OS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Search Results (1):\n- **Name:** The Tower\n- **Category:** Landmark\n- **Has Parking:** True\n- **Distance to Central Plaza:** 0.0', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1731656178, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_467dc6e35f', usage=CompletionUsage(completion_tokens=36, prompt_tokens=3254, total_tokens=3290, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, acce

Initialized parallel conversations:

Stream 1:

Initial response:  Thanks for your response! Central Plaza and The Mall will certainly be part of our plan. A...

Features extracted: {}

...

Initial response:  Search(fields=, filters=[category ==...

Features extracted: {}

Stream 3:

Initial response:  Search(fields=, filters=, sort_by=[distanc...

Features extracted: {}

In [160]:
for conv in parallel_convs.conversations:
    print(conv.players['agent'].strategy)


1. Clarification and Prioritization: Begin by confirming the user's top priorities among all preferences listed. 
Specifically ask them to rank the importance of landmarks, Dan's recommendations, live music, parking, vegan 
options, outdoor seating, and kid-friendliness to ensure the final itinerary meets their most critical needs.

2. Use of Tools for Landmark Search: Utilize available tools or databases to quickly search and suggest landmarks 
that also meet the user's other preferences such as live music, vegan options, and budget constraints. This will 
help provide a recommendation efficiently and accurately.

3. Conciseness and Confirmation: After suggesting a landmark that fits the user's criteria, summarize the proposed 
itinerary to the user, including Central Plaza, The Mall, and the recommended landmark. Confirm that these choices 
align with their preferences and budget before finalizing the itinerary to ensure satisfaction.

In [174]:
style_response = style_judger(parallel_convs)
parallel_convs = step_parallel_conversations(parallel_convs)

user_conversations_no_pref = clean_user_conversation(user.user_prompt_obss, parallel_convs)

----Conversation 0 is waiting for user input---

Response:  [think] The Tower seems to be a good third option since it matches with the user's preference for a 
landmark and it's conveniently close to Central Plaza. Let's propose this itinerary.

stop:  stop

CompletionUsage(
    completion_tokens=38,
    prompt_tokens=2276,
    total_tokens=2314,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

Response:  [propose] [Central Plaza, The Mall, The Tower]

...

Last observation: 
Agent:  Thanks for your response! Central Plaza and The Mall will certainly be part of our...

Conversation 1 (Turn 9):

Last observation: 
Agent:  Here's what I propose: start by visiting Central Plaza and The Mall, then take a s...

Conversation 2 (Turn 8):

Last observation: 
Agent:  I have a recommendation for you that aligns with your preferences: Central Plaza a...

In [175]:
def initialize_10_user_conversations(parallel_convs, user_conversations_no_pref, style_response, extracted_features, reset_sonlist = False):
    if reset_sonlist:
        for conversation in parallel_convs.conversations:
            conversation.son_convs = []
    # we want 10 user personas per conversation; they should be the 10 sam ones but copies 
    n_10_versions = {}
    for i in range(10):
        # any conversation is fine since action log is not counted
        env_copy = copy.deepcopy(parallel_convs.conversations[0].env)
        game_state = format_user_game_state(env_copy, incomplete_action_log = True)
        #parsed_features = parse_extracted_features(extracted_features)
        obss = env_copy.reset(game_state, known_user_preferences_num=(10-len(extracted_features.keys())), extracted_features = extracted_features)
        bundle = {'env': env_copy, 'obss': obss}
        n_10_versions[i] = bundle
    
    # user and agent are using different games; user needs to be new, agent is not 
    for conv_idx, conversation in enumerate(parallel_convs.conversations):
        base_players = parallel_convs.players_dict[conv_idx]
        base_chat_history = user_conversations_no_pref[conv_idx]
        for a_bundle in range(len(n_10_versions)):
            env_copy = copy.deepcopy(n_10_versions[a_bundle]['env'])
            env_copy.game.action_log = copy.deepcopy(conversation.env.game.action_log)
            obss = copy.deepcopy(n_10_versions[a_bundle]['obss'])
            #print(obss['user'])
            #extracted_features = parse_extracted_features(extracted_features)
            n_1_prompts = n_1_user_personas(obss['user'], extracted_features, record = base_chat_history, style_response = style_response)
            players = parallel_convs.create_pair_new_players(base_players, user_personas_incomplete_prompt=n_1_prompts)
            new_conv = Conversation(players=players, env=env_copy, max_length=parallel_convs.max_length, parent_conv=conversation)
            conversation.son_convs.append(new_conv)
            #new_conv = Conversation(players=pairs_3_10_2_pairs_dict[conv_idx][f'pair_{i}'], env=env_copy, max_length=parallel_convs.max_length, parent_conv=conversation)
            #conversation.son_convs.append(new_conv)
    global global_preferences_only
    return parallel_convs, global_preferences_only
        
        
        
        
  

In [176]:
parallel_convs, preferences_only = initialize_10_user_conversations(parallel_convs, user_conversations_no_pref, style_response, extracted_features, reset_sonlist = True)
#print(extracted_features)

..searching with query:  {'fields': ['name'], 'filters': [['category', '==', 'landmark']]}
..searching with query:  {'fields': ['name'], 'filters': [['category', '==', 'concert']]}
..searching with query:  {'fields': ['name'], 'text_query': 'live music'}
..searching with query:  {'fields': ['name', 'price'], 'text_query': 'live music', 'filters': [['price', '<=', '40']]}
..searching with query:  {'fields': ['name', 'price'], 'filters': [['category', '==', 'restaurant'], ['price', '<=', '10']], 'sort_by': [['distance_to', 'The Mall']]}
..searching with query:  {'fields': ['name', 'price', 'distance'], 'filters': [['category', '==', 'restaurant']], 'sort_by': [['distance_to', 'The Mall'], 'price']}
..searching with query:  {'fields': ['name'], 'text_query': 'good for kids', 'filters': [['category', '==', 'park']], 'sort_by': [['distance_to', "Saul's"]]}
..searching with query:  {'fields': ['name'], 'filters': [['vegan', '==', 'true']]}
..searching with query:  {'fields': ['name'], 'filte

In [177]:
for conv in parallel_convs.conversations:
    print(conv.players['agent'].strategy)
    for son_conv in conv.son_convs:
        print(son_conv.players['agent'].strategy)



1. **Confirm Priorities and Deal Breakers**: Reiterate the user's confirmation that parking is not a deal breaker 
and prioritize Central Plaza and The Mall. Ask if there are any other preferences or deal breakers, such as 
specific landmarks or types of activities, to ensure a comprehensive understanding of the user's priorities.

1. **Confirm Priorities and Deal Breakers**: Reiterate the user's confirmation that parking is not a deal breaker 
and prioritize Central Plaza and The Mall. Ask if there are any other preferences or deal breakers, such as 
specific landmarks or types of activities, to ensure a comprehensive understanding of the user's priorities.

1. **Confirm Priorities and Deal Breakers**: Reiterate the user's confirmation that parking is not a deal breaker 
and prioritize Central Plaza and The Mall. Ask if there are any other preferences or deal breakers, such as 
specific landmarks or types of activities, to ensure a comprehensive understanding of the user's priorities.

1. **Confirm Priorities and Deal Breakers**: Reiterate the user's confirmation that parking is not a deal breaker 
and prioritize Central Plaza and The Mall. Ask if there are any other preferences or deal breakers, such as 
specific landmarks or types of activities, to ensure a comprehensive understanding of the user's priorities.

1. **Confirm Priorities and Deal Breakers**: Reiterate the user's confirmation that parking is not a deal breaker 
and prioritize Central Plaza and The Mall. Ask if there are any other preferences or deal breakers, such as 
specific landmarks or types of activities, to ensure a comprehensive understanding of the user's priorities.

...

3. **Summarize and Confirm Final Itinerary**: Summarize the proposed itinerary, including Central Plaza, The Mall, 
and the additional location with parking. Confirm that this meets the user's expectations and ask if there are any 
last-minute changes or additional preferences before finalizing the itinerary.

3. **Summarize and Confirm Final Itinerary**: Summarize the proposed itinerary, including Central Plaza, The Mall, 
and the additional location with parking. Confirm that this meets the user's expectations and ask if there are any 
last-minute changes or additional preferences before finalizing the itinerary.

3. **Summarize and Confirm Final Itinerary**: Summarize the proposed itinerary, including Central Plaza, The Mall, 
and the additional location with parking. Confirm that this meets the user's expectations and ask if there are any 
last-minute changes or additional preferences before finalizing the itinerary.

3. **Summarize and Confirm Final Itinerary**: Summarize the proposed itinerary, including Central Plaza, The Mall, 
and the additional location with parking. Confirm that this meets the user's expectations and ask if there are any 
last-minute changes or additional preferences before finalizing the itinerary.

3. **Summarize and Confirm Final Itinerary**: Summarize the proposed itinerary, including Central Plaza, The Mall, 
and the additional location with parking. Confirm that this meets the user's expectations and ask if there are any 
last-minute changes or additional preferences before finalizing the itinerary.

In [82]:
print(parallel_convs.conversations[2].history)
#print(global_preferences_only)



[
    ConversationState(
        obss={
            'user': '',
            'agent': "[tool] Search(fields=[name, category, good for kids, outdoor seating, has parking], 
filters=[category == landmark])\nSearch Results (4):\n- name: Einstein's summer house\n  category: landmark\n  good
for kids: True\n  outdoor seating: Not specified\n  has parking: False",
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=9,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': '',
            'agent': "[name, category, good for kids, outdoor seating, has parking]\nError: Invalid message: [name,
category, good for kids, outdoor seating, has parking].Messages must be formatted with a type like 
'[message]<content here>'",
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=10,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': '',
            'agent': "I've found one landmark, Einstein's summer house, that could be of interest. However, it does
not specify outdoor seating and it doesn't have parking. Would the lack of parking and unspecified outdoor seating 
be a deal-breaker for you?\nError: Invalid message: I've found one landmark, Einstein's summer house, that could be
of interest. However, it does not specify outdoor seating and it doesn't have parking. Would the lack of parking 
and unspecified outdoor seating be a deal-breaker for you?.Messages must be formatted with a type like 
'[message]<content here>'",
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=11,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': "\nAgent: [message] I've found a landmark called Einstein's summer house that could be of 
interest. However, it doesn't have parking and doesn't specify if there is outdoor seating. Would these factors be 
deal-breakers for you?",
            'agent': '',
            'turn_player': 'user',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=12,
        responses=None,
        player_states=None
    )
]

In [83]:
def step_all_user_responses( parallel_convs):
    """
    For each of the 3 conversation paths:
        - Get all 10 users to respond to their agent
        - Step the environment with each response
    """
    
    all_states = []
    all_envs = []
    
    # For each conversation path (0,1,2)
    for conversation in parallel_convs.conversations:
        if conversation.propose_made:
            continue
        envs = []
        conv_states = []
   
        current_state = conversation.get_current_state()
        # this get the obss of 3 main branches 
        
        
        
        
        # Get responses from all 10 users for this conversation
        for son_conv in conversation.son_convs:
            user = son_conv.players['user']
            
            
            ##state = conversation.reverse_state()
            ##print(state.obss)
            ##break
            
            
            
            # Have user observe current state
            #user.observe(current_state.obss['user'])
            
            # Get user's response
            while True:
                user_response = user.respond(t = current_state.turn, max_len = parallel_convs.max_length, vary = True)
                new_state = son_conv.step(user_response, current_state.turn + 1)
                if new_state.obss["turn_player"] == 'agent':
                    break
                user.observe(new_state.obss['user'])
            
            # Step environment with user response
                
            print(son_conv.env.game.turn_player)
            conv_states.append(new_state)
            
            # Have agent observe the new state
        
        all_states.append(conv_states)
    
    return all_states

# Use the function


# Print some results to verify


In [178]:

conversation_states = step_all_user_responses( parallel_convs)

Response:  [message] Other than that, it'd be great if the places can be reserved in advance and are less crowded, 
away from the typical tourist traps. My budget is also $30, so if we could stay under that, that'd be ideal. I'm 
not a big fan of American, Ethiopian, French, Chinese, Spanish, Kosher, or seafood cuisine.

stop:  stop

CompletionUsage(
    completion_tokens=75,
    prompt_tokens=2335,
    total_tokens=2410,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

1

Response:  [message] I would love to find a place with a wifi where I can work for a bit and preferably with vegan 
options. Ratings above 4 would be ideal. As we are also quite conscious about COVID, outdoor seating would be best.
And although I am traveling in a group of 7, group suitability is not very important. Finally, as always, staying 
below budget is important too.

...

1

Response:  [message] Not really, I'd just prefer places that aren't too loud with live music. Also, if possible, 
try to keep the travel distance between the landmarks to a minimum.

stop:  stop

CompletionUsage(
    completion_tokens=38,
    prompt_tokens=2297,
    total_tokens=2335,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

1

In [179]:
print(len(conversation_states[0]))


10

In [181]:
def run_until_agent_proposals(parallel_convs):
    """
    For each conversation path:
        - Allow agent to tool call or think multiple times
        - Continue until agent makes a proposal
        - Reject regular messages - only allow tool calls, thinks, or proposals
    """
    all_proposals = []
    
    # For each conversation path (0,1,2)
    for conv_idx, conversation in enumerate(parallel_convs.conversations):
        if conversation.propose_made:
            continue
        path_proposals = []
        
        # For each son conversation
        for son_conv in conversation.son_convs:
            agent = son_conv.players['agent']
            agent.observe(son_conv.state.obss['agent'])
            
            # Keep getting responses until we get a proposal
            must_propose = False
            while True:
                response = agent.respond(t=son_conv.state.turn + 1, vary=True, propose=must_propose)
                if '[tool]' in response or '[think]' in response:
                    # Valid tool/think response
                    state = son_conv.step(response, son_conv.state.turn + 1)
                    agent.observe(state.obss['agent'])
                    continue
                if '[message]' in response:
                    must_propose = True
                    continue
                state = son_conv.step(response, son_conv.state.turn + 1, propose = must_propose)
                # Check response type
                if (must_propose or ('[propose]' in state.obss['agent'])) and 'Error' not in state.obss['agent']:
                    # Got a proposal - save it and move on
                    #son_conv.step(response, son_conv.state.turn + 1, propose=True)
                    path_proposals.append(response)
                    break
                agent.observe(state.obss['agent'])

                
            print(f"Final proposal: {response}")
            
        all_proposals.append(path_proposals)
    
    return all_proposals

In [88]:
def extract_total_score(obss_string):
    # Find the line with "TOTAL SCORE:"
    for line in obss_string.split('\n'):
        if "TOTAL SCORE:" in line:
            # Extract the number after the equals sign
            score = int(line.split('=')[-1])
            return score
    return None


In [182]:
agent_proposals = run_until_agent_proposals(parallel_convs)


Response:  [message] Great, we'll certainly incorporate Central Plaza and The Mall into the itinerary. It sounds 
like being able to reserve in advance and avoiding crowded places are important to you. With regard to your third 
destination, are there any particular landmarks you're interested in? Also, do you have a preference for indoor or 
outdoor activities?

stop:  stop

CompletionUsage(
    completion_tokens=66,
    prompt_tokens=2368,
    total_tokens=2434,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

Response:  [Central Plaza, The Mall, NULL]

stop:  stop

...

CompletionUsage(
    completion_tokens=48,
    prompt_tokens=2447,
    total_tokens=2495,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

Response:  [Central Plaza, The Mall, NULL]

stop:  stop

CompletionUsage(
    completion_tokens=9,
    prompt_tokens=2391,
    total_tokens=2400,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

Final proposal: [Central Plaza, The Mall, NULL]

In [187]:
for conv in parallel_convs.conversations:
    print(conv.history)
    print(0)


[
    ConversationState(
        obss={
            'user': "\nAgent: [message] Thanks for your response! Central Plaza and The Mall will certainly be part
of our plan. Are there any specific landmarks or activities you'd like to consider in our plan, or other 
deal-breakers to keep in mind?",
            'agent': '',
            'turn_player': 'user',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=6,
        responses=None,
        player_states=None
    )
]

0

[
    ConversationState(
        obss={
            'user': '',
            'agent': '[tool] Search(fields=[name, category, has parking, distance_to(Central Plaza)], 
filters=[category == landmark, has parking == True], sort_by=[distance_to(Central Plaza)], limit=1)\nSearch Results
(1):\n- **Name:** The Tower\n- **Category:** Landmark\n- **Has Parking:** True\n- **Distance to Central Plaza:** 
0.0',
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=6,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': '',
            'agent': "[think] The Tower seems to be a good third option since it matches with the user's preference
for a landmark and it's conveniently close to Central Plaza. Let's propose this itinerary.",
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=7,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': "\nAgent: [propose] [Central Plaza, The Mall, The Tower]\nProposal Score:\n1) (score: 3) 
Central Plaza\ngood for groups: True\nhas parking: False\nopen late: True\nrating: 5\ntouristy: False\n2) (score: 
-6) Travel from Central Plaza to The Mall, 1.0mi\n3) (score: 6) The Mall\ngood for groups: False\ngood for kids: 
True\nopen late: True\nrating: 3\ntouristy: True\n4) (score: 0) Travel from The Mall to The Tower, 0.1mi\n5) 
(score: -6) The Tower\ngood for groups: True\ngood for kids: False\nhas parking: True\nopen late: False\ntouristy: 
True\n\nOverall Checklist:\nYES (score: 9) go to at least one landmark\nYES (score: 4) definitely want to check out
Dan's recommendations: Central Plaza, The Mall\nNO (score: -4) keep budget below $40\nTOTAL SCORE: 
+3-6+6+0-6+9+4-4=6\nYou can [think], or output one of these choices:\n(1) [accept] Accept\n(2) [reject] Reject",
            'agent': '[propose] [Central Plaza, The Mall, The Tower]',
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=8,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': "\nAgent: [message] Here's what I propose: start by visiting Central Plaza and The Mall, then 
take a short walk to The Tower, a nearby landmark which offers parking in case you need it. What do you think about
this plan?",
            'agent': '',
            'turn_player': 'user',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 4}
        },
        features={},
        turn=9,
        responses=None,
        player_states=None
    )
]

0

[
    ConversationState(
        obss={
            'user': '',
            'agent': '[tool] Search(fields=[name, category, has parking], filters=[category == landmark], 
sort_by=[distance_to(The Mall)], limit=1)\nSearch Results (1):\n- **Name:** The Tower\n- **Category:** Landmark\n- 
**Has Parking:** True',
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=6,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': "\nAgent: [propose] [Central Plaza, The Mall, The Tower]\nProposal Score:\n1) (score: 3) 
Central Plaza\ngood for groups: True\nhas parking: False\nopen late: True\nrating: 5\ntouristy: False\n2) (score: 
-6) Travel from Central Plaza to The Mall, 1.0mi\n3) (score: 6) The Mall\ngood for groups: False\ngood for kids: 
True\nopen late: True\nrating: 3\ntouristy: True\n4) (score: 0) Travel from The Mall to The Tower, 0.1mi\n5) 
(score: -6) The Tower\ngood for groups: True\ngood for kids: False\nhas parking: True\nopen late: False\ntouristy: 
True\n\nOverall Checklist:\nYES (score: 9) go to at least one landmark\nYES (score: 4) definitely want to check out
Dan's recommendations: Central Plaza, The Mall\nNO (score: -4) keep budget below $40\nTOTAL SCORE: 
+3-6+6+0-6+9+4-4=6\nYou can [think], or output one of these choices:\n(1) [accept] Accept\n(2) [reject] Reject",
            'agent': '[propose] [Central Plaza, The Mall, The Tower]',
            'turn_player': 'agent',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 3}
        },
        features={},
        turn=7,
        responses=None,
        player_states=None
    ),
    ConversationState(
        obss={
            'user': "\nAgent: [message] I have a recommendation for you that aligns with your preferences: Central 
Plaza and The Mall, which you've mentioned before, as well as The Tower - it's a landmark and it has parking. What 
do you think of this itinerary?",
            'agent': '',
            'turn_player': 'user',
            'done': False,
            'reward': 0,
            'info': {'num_msgs': 4}
        },
        features={},
        turn=8,
        responses=None,
        player_states=None
    )
]

0

In [185]:
print(agent_proposals)

[
    [
        '[Central Plaza, The Mall, NULL]',
        '[Central Plaza, The Mall, NULL]',
        '[Central Plaza, The Mall, Garden of Wonders]',
        '[Central Plaza, The Mall, NULL]',
        '[Central Plaza, The Mall, Riverside Terrace]',
        '[Central Plaza, The Mall, Hindenberg Memorial]',
        '[Central Plaza, The Mall, Liberty Memorial]',
        '[Central Plaza, The Mall, Atlas Park]',
        '[Central Plaza, The Mall, The Dockside Grill]',
        '[Central Plaza, The Mall, NULL]'
    ]
]

In [18]:
def hardcode_rate_agent_proposals(agent_proposals, parallel_convs):
    proposal_scores = []
    for conversation in parallel_convs.conversations:
        scores = []
        for son_conv in conversation.son_convs:
            score = extract_total_score(son_conv.state.obss['user'])
            normalized_score = son_conv.env._normalize(score)
            scores.append(round(normalized_score, 3))
            #print(normalized_score)
        proposal_scores.append(scores)
    return proposal_scores
            
            





In [19]:
def get_tool_calls(conversation):
    tool_calls = []
    for state in conversation.history:
        if state.obss['agent'].startswith('[tool]'):
            tool_calls.append(state.obss['agent'])
    tool_calls = '\n'.join(tool_calls)
    return tool_calls




In [20]:
def api_score_extraction(response):
    try:
        score = int(response.split('TOTAL SCORE:')[-1].strip())
    except:
        # Extract just the number after TOTAL SCORE:
        score = []
        letters = list(response.split('TOTAL SCORE:')[-1].strip())
        for letter in letters:
            if letter.isdigit():
               score.append(letter)
        score = int(''.join(score))
    return score


In [191]:
def extract_proposal(conversation):
    for state in conversation.history:
        if '[propose]' in state.obss['agent'] and 'Error' not in state.obss['agent']:
            proposal = state.obss['agent']
    return proposal


In [195]:
def api_rate_agent_proposals(agent_proposals, parallel_convs):
    # here can use the global_preferences_only
    all_prompts = []
    USER_EVAL_PROMPT = """You are a user evaluating an AI travel agent's itinerary proposals. You have specific preferences, and you should evaluate how well the agent's proposals match these preferences.

Your preferences are:
{preferences}

For each proposal, you will see:
1. The proposed itinerary
2. The agent's search history showing attributes of each location

The proposal is:
{proposal}

The search history is:
{tool_calls}

Please evaluate each proposal by:
1. Reading the proposal and search history carefully
2. Checking if the locations match your stated preferences
3. output a score between 0 and 100 in the format of 'TOTAL SCORE: 100'"""
    global global_preferences_only
    all_tool_calls = []
    count = 0
    all_scores = []
    for i, conversation in enumerate(parallel_convs.conversations):
        tool_calls = get_tool_calls(conversation)
        all_tool_calls.append(tool_calls)
        if conversation.propose_made:
            proposal = extract_proposal(conversation)
        else:
            proposal_list = agent_proposals[i]
        prompts = []
        scores = []
        i = 0
        while i < 10:
            if not conversation.propose_made:
                proposal = proposal_list[i]
            preferences = global_preferences_only[count%10]
            tailored_prompt = USER_EVAL_PROMPT.format(preferences = preferences, proposal = proposal, tool_calls = tool_calls)
            response = Agent_tell(tailored_prompt)
            try:
                score = api_score_extraction(response)
            except:
                score = -1
            #print(score)
            if 0 <= score <= 100:
                count += 1
                scores.append(score)
                prompts.append(tailored_prompt)
                i += 1
        all_scores.append(scores)
        all_prompts.append(prompts)
    return all_scores, all_prompts
            
   





In [22]:
def choose_best_path(scores):
    best_score = 0
    best_path = None
    for i, score_list in enumerate(scores):
        path_average = sum(score_list) / len(score_list) if score_list else 0
        if path_average > best_score:
            best_score = path_average
            best_path = i
    return best_path




In [23]:
def save_best_path_as_checkpoint(parallel_convs, best_path, checkpoint_manager):
    """Save the best conversation path as a checkpoint"""
    best_conv = parallel_convs.conversations[best_path]
    
    
    # Create state dict from best conversation
    state = {
        'players': best_conv.players,
        'history': best_conv.history,
        'env_state': best_conv.env.game.get_game_info(override_events=True),
        'turn': best_conv.history[-1].turn if best_conv.history else 0
    }
    
    # Save as checkpoint 
    checkpoint_name = f"best_path_{int(time.time())}"
    checkpoint_manager.save_state(state, checkpoint_name)
    return checkpoint_name





In [196]:
scores = api_rate_agent_proposals(agent_proposals, parallel_convs)
best_path = choose_best_path(scores)


In [197]:
print(best_path)
print(scores)

2

[
    [40, 70, 20, 50, 10, 85, 60, 30, 50, 50],
    [40, 60, 50, 50, 60, 70, 40, 40, 50, 80],
    [50, 60, 50, 70, 50, 70, 60, 60, 50, 60]
]

In [35]:

checkpoint_mgr = CheckpointManager()

        # Load the initial state
initial_state = checkpoint_mgr.load_full_state("initial_state")
          
        
    
    
    
 
    
try:
    # Access key components
    players = initial_state['players']
    extracted_features = initial_state.get('extracted_features', [])
    t = initial_state.get('t', 0)
    env = initial_state.get('env', None)
    agent = initial_state['players']['agent']
    user = initial_state['players']['user']
    chat_history = agent.prompt
    if not env:
        print("No environment found in checkpoint.")
    print(f"\nTurn: {t}")
    print(f"Features: {extracted_features}")
    print(f"Players: {list(players.keys())}")
    
except Exception as e:
    print(f"Error loading checkpoint: {str(e)}")


agent_3_responses = n_different_responses(players, t = t)

parallel_convs = initialize_parallel_conversations(agent_3_responses, players, env, t)
style_response = style_judger(parallel_convs)
parallel_convs = step_parallel_conversations(parallel_convs)

user_conversations_no_pref = clean_user_conversation(user.user_prompt_obss, parallel_convs)
########################
parallel_convs, preferences_only = initialize_10_user_conversations(parallel_convs, user_conversations_no_pref, style_response, extracted_features, reset_sonlist = True)

extracted_features = parse_extracted_features(extracted_features)
conversation_states = step_all_user_responses( parallel_convs)

agent_proposals = get_agent_proposals( parallel_convs)
###########
scores = api_rate_agent_proposals(agent_proposals, parallel_convs)
best_path = choose_best_path(scores)
checkpoint_name = save_best_path_as_checkpoint(parallel_convs, best_path, checkpoint_mgr)
print(f"Successfully processed checkpoint. Saved as: {checkpoint_name}")

Turn: 13

Features: ['', '```json\n{\n  "live music": "like",\n  "touristy": "dislike",\n  "good for kids": true,\n  "good 
for groups": true,\n  "has_parking": true\n}\n```', '', '', '', '', '', '', '', '', '', '', '{}']

Players: ['agent', 'user']

Response:  [propose] [Schwarzes Cafe, Cookies Cream, Caribbean Corner]

stop:  stop

...

0

Response:  [accept]

stop:  stop

CompletionUsage(
    completion_tokens=3,
    prompt_tokens=2742,
    total_tokens=2745,
    completion_tokens_details=CompletionTokensDetails(
        reasoning_tokens=0,
        audio_tokens=0,
        accepted_prediction_tokens=0,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}
)

> /Users/georgiazhou/research_machine/dialop/dialop/envs/planning.py(184)step()
    182                     type_ = "accept"
    183                 import pdb; pdb.set_trace()
--> 184                 done, game_infos = self._proposal_response(
    185                     type_ == "accept",
    186                     self.game.turn_player)

!!! Traceback (most recent call last):
  File "/Users/georgiazhou/research_machine/dialop/dialop/notebooks/../envs/planning.py", line 184, in step
    type_ == "accept",
  File "/Users/georgiazhou/research_machine/dialop/dialop/notebooks/../envs/planning.py", line 184, in step
    type_ == "accept",
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/bdb.py", line 88, in trace_dispatch
    return self.dispatch_line(frame)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/bdb.py", line 113, in dispatch_line
    if self.quitting: raise Bd